# AutoGluon MultimodalPredictor for House Price Prediction

In this notebook, we will use AutoGluon's `MultimodalPredictor` to create a machine learning model for predicting house prices.
This dataset contains both structured (numerical and categorical) and unstructured (text) data, and AutoGluon allows us to easily handle such datasets using its powerful AutoML capabilities.
We will follow these steps:
1. Install AutoGluon
2. Preprocess the data
3. Train a model using AutoGluon's MultimodalPredictor
4. Evaluate and save the model's predictions
    

## Step 1: Install AutoGluon

In [2]:
!pip install autogluon.multimodal

  Using cached autogluon.multimodal-1.1.1-py3-none-any.whl.metadata (12 kB)
  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached boto3-1.35.31-py3-none-any.whl.metadata (6.6 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached lightning-2.3.3-py3-none-any.whl.metadata (35 kB)
  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
  Using cached accelerate-0.21.0-py3-none-any.whl.metadata (17 kB)
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached timm-0.9.16-py3-none-any.whl.metadata (38 kB)
  Using cached torchvision-0.18.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)

In [ ]:
!pip install autogluon
!pip install torch torchvision torchaudio transformers

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.1
    Uninstalling torchaudio-2.4.1:
      Successfully uninstalled torchaudio-2.4.1


In [ ]:
!pip uninstall torch torchaudio -y
!pip install torch torchvision torchaudio

Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: torchaudio 2.3.1
Uninstalling torchaudio-2.3.1:
  Successfully uninstalled torchaudio-2.3.1
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchaudio-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached triton-3.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Using cached torchaudio-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached torchaudio-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
Using cached torc

In [ ]:
!pip install autogluon --upgrade

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
#import data from local machine
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


## Step 2: Preprocessing the Data

In [ ]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# Check for missing values
missing_train = train_data.isnull().sum()
print("Missing values in train data:", missing_train)

# Handle missing values by filling with median for numerical and mode for categorical/text
for col in train_data.columns:
    if train_data[col].dtype in ['float64', 'int64']:
        train_data[col].fillna(train_data[col].median(), inplace=True)
    else:
        train_data[col].fillna(train_data[col].mode()[0], inplace=True)

# Repeat for test data
for col in test_data.columns:
    if test_data[col].dtype in ['float64', 'int64']:
        test_data[col].fillna(test_data[col].median(), inplace=True)
    else:
        test_data[col].fillna(test_data[col].mode()[0], inplace=True)

# Ensure the target column ('Sold Price') is separate
train_data.head()


Missing values in train data: Id                                 0
Address                            0
Sold Price                         0
Summary                          354
Type                               0
Year built                      1045
Heating                         6852
Cooling                        20694
Parking                         1374
Lot                            14181
Bedrooms                        2872
Bathrooms                       3465
Full bathrooms                  7865
Total interior livable area     2526
Total spaces                     916
Garage spaces                    917
Region                             2
Elementary School               4742
Elementary School Score         4896
Elementary School Distance      4742
Middle School                  16704
Middle School Score            16705
Middle School Distance         16704
High School                     5000
High School Score               5219
High School Distance            5001
Flooring

,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,2017-06-30,598000.0,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,Central Air,0 spaces,9147.0,...,"Garage, Garage - Attached, Covered",49627.0,468.0,2019-08-25,180000.0,2017-06-30,598000.0,Strawberry,95375,CA
3,3,10750 Braddock Dr,1775000.0,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",6502.0,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,4,7415 O Donovan Rd,1175000.0,Beautiful 200 acre ranch land with several pas...,VacantLand,1967.0,Central,Central Air,0 spaces,6502.0,...,"Garage, Garage - Attached, Covered",547524.0,7129.0,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


In [ ]:
# Print the column names to verify if 'Sold Price' exists
print(train_data.columns)

Index(['Id', 'Address', 'Sold Price', 'Summary', 'Type', 'Year built',
       'Heating', 'Cooling', 'Parking', 'Lot', 'Bedrooms', 'Bathrooms',
       'Full bathrooms', 'Total interior livable area', 'Total spaces',
       'Garage spaces', 'Region', 'Elementary School',
       'Elementary School Score', 'Elementary School Distance',
       'Middle School', 'Middle School Score', 'Middle School Distance',
       'High School', 'High School Score', 'High School Distance', 'Flooring',
       'Heating features', 'Cooling features', 'Appliances included',
       'Laundry features', 'Parking features', 'Tax assessed value',
       'Annual tax amount', 'Listed On', 'Listed Price', 'Last Sold On',
       'Last Sold Price', 'City', 'Zip', 'State'],
      dtype='object')


In [ ]:
print(test_data.columns)

Index(['Id', 'Address', 'Summary', 'Type', 'Year built', 'Heating', 'Cooling',
       'Parking', 'Lot', 'Bedrooms', 'Bathrooms', 'Full bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Region', 'Elementary School', 'Elementary School Score',
       'Elementary School Distance', 'Middle School', 'Middle School Score',
       'Middle School Distance', 'High School', 'High School Score',
       'High School Distance', 'Flooring', 'Heating features',
       'Cooling features', 'Appliances included', 'Laundry features',
       'Parking features', 'Tax assessed value', 'Annual tax amount',
       'Listed On', 'Listed Price', 'Last Sold On', 'Last Sold Price', 'City',
       'Zip', 'State'],
      dtype='object')


## Step 3: Setting Up AutoGluon's MultimodalPredictor

In [ ]:
from autogluon.multimodal import MultiModalPredictor

# Define the target variable
target_column = 'Sold Price'

# Initialize the predictor
predictor = MultiModalPredictor(label=target_column, problem_type='regression')

# Train the model using the training data
predictor.fit(train_data=train_data, time_limit=600)  # Time limit of 10 minutes for training


No path specified. Models will be saved in: "AutogluonModels/ag-20241001_053638"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.3.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       9.80 GB / 12.67 GB (77.4%)
Disk Space Avail:   58.38 GB / 107.72 GB (54.2%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20241001_053638
    ```

INFO: Seed set to 0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to f

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:10:14. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 1: 'val_rmse' reached 1.40416 (best 1.40416), saving model to '/content/AutogluonModels/ag-20241001_053638/epoch=0-step=1.ckpt' as top 3
AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20241001_053638")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




## Step 4: Evaluating the Model

In [ ]:
# Ensure you have predictions for the test dataset
predictions = predictor.predict(test_data)

# Prepare the submission file
submission = pd.DataFrame({'Id': test_data['Id'], 'Sold Price': predictions})
submission.to_csv('submission.csv', index=False)

# Verify the first few rows of the submission file
print(submission.head())


Predicting: |          | 0/? [00:00<?, ?it/s]